In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from adahessian import AdaHessian

In [2]:
import time

In [3]:
epochs = 10
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()
use_cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
Network = Net().to(device)
#Network = Net()

In [7]:
Network.train()
vw0_model = Network
vw1_model = Network
vw2_model = Network
vw3_model = Network
vw4_model = Network
vw5_model = Network
vw6_model = Network
vw7_model = Network
vw8_model = Network
vw9_model = Network

In [8]:
#vw0_optimizer = optim.SGD(vw0_model.parameters(), lr=args.lr)
#vw1_optimizer = optim.SGD(vw1_model.parameters(), lr=args.lr)
#vw2_optimizer = optim.SGD(vw2_model.parameters(), lr=args.lr)
#vw3_optimizer = optim.SGD(vw3_model.parameters(), lr=args.lr)
#vw4_optimizer = optim.SGD(vw4_model.parameters(), lr=args.lr)
#vw5_optimizer = optim.SGD(vw5_model.parameters(), lr=args.lr)
#vw6_optimizer = optim.SGD(vw6_model.parameters(), lr=args.lr)
#vw7_optimizer = optim.SGD(vw7_model.parameters(), lr=args.lr)
#vw8_optimizer = optim.SGD(vw8_model.parameters(), lr=args.lr)
#vw9_optimizer = optim.SGD(vw9_model.parameters(), lr=args.lr)
vw0_optimizer = AdaHessian(vw0_model.parameters(), lr=args.lr)
vw1_optimizer = AdaHessian(vw1_model.parameters(), lr=args.lr)
vw2_optimizer = AdaHessian(vw2_model.parameters(), lr=args.lr)
vw3_optimizer = AdaHessian(vw3_model.parameters(), lr=args.lr)
vw4_optimizer = AdaHessian(vw4_model.parameters(), lr=args.lr)
vw5_optimizer = AdaHessian(vw5_model.parameters(), lr=args.lr)
vw6_optimizer = AdaHessian(vw6_model.parameters(), lr=args.lr)
vw7_optimizer = AdaHessian(vw7_model.parameters(), lr=args.lr)
vw8_optimizer = AdaHessian(vw8_model.parameters(), lr=args.lr)
vw9_optimizer = AdaHessian(vw9_model.parameters(), lr=args.lr)

In [9]:
compute_nodes = [0]*10

In [10]:
remote_dataset = (list(),list(),list(),list(),list(),list(),list(),list(),list(),list())

for batch_idx, (data,target) in enumerate(train_loader):
    data = data.to(device)
    target = target.to(device)
    remote_dataset[batch_idx % len(compute_nodes)].append((data, target))

In [11]:
models = [vw0_model, vw1_model, vw2_model, vw3_model, vw4_model, vw5_model, vw6_model, vw7_model, vw8_model, vw9_model]
params = [list(vw0_model.parameters()), list(vw1_model.parameters()), list(vw2_model.parameters()),
          list(vw3_model.parameters()), list(vw4_model.parameters()), list(vw5_model.parameters()),
          list(vw6_model.parameters()), list(vw7_model.parameters()), list(vw8_model.parameters()),
          list(vw9_model.parameters())]

optimizers = [vw0_optimizer, vw1_optimizer, vw2_optimizer, vw3_optimizer, vw4_optimizer, vw5_optimizer, vw6_optimizer,
              vw7_optimizer, vw8_optimizer, vw9_optimizer]

# Federated averaging

In [12]:
def train(epoch, Network, averaging_steps):    
    Network.train()
    for data_index in range(len(remote_dataset[0])-1):        
        for remote_index in range(len(compute_nodes)):
            data, target = remote_dataset[remote_index][data_index]
            models[remote_index] = Network
            #optimizers[remote_index] = optim.SGD(models[remote_index].parameters(), lr=args.lr)
            optimizers[remote_index] = AdaHessian(models[remote_index].parameters(), lr=args.lr)
            for i in range(averaging_steps):
                optimizers[remote_index].zero_grad()
                pred = models[remote_index](data)
                loss = F.nll_loss(pred, target)
                loss.backward(create_graph = True)
                optimizers[remote_index].step()
        with torch.no_grad():
            temp1, temp2, temp3, temp4, temp5, temp6, temp7, temp8 = 0, 0, 0, 0, 0, 0, 0, 0
            for i in range(len(compute_nodes)):
                temp1 += models[i].conv1.weight.data
                temp2 += models[i].conv1.bias.data
                temp3 += models[i].conv2.weight.data
                temp4 += models[i].conv2.bias.data
                temp5 += models[i].fc1.weight.data
                temp6 += models[i].fc1.bias.data
                temp7 += models[i].fc2.weight.data
                temp8 += models[i].fc2.bias.data
            Network.conv1.weight.set_(temp1/10)
            Network.conv1.bias.set_(temp2/10)
            Network.conv2.weight.set_(temp3/10)
            Network.conv2.bias.set_(temp4/10)
            Network.fc1.weight.set_(temp5/10)
            Network.fc1.bias.set_(temp6/10)
            Network.fc2.weight.set_(temp7/10)
            Network.fc2.bias.set_(temp8/10)

In [13]:
def test():
    for i in range(1):
        Network.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = Network(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

In [14]:
averaging_steps = 5
for epoch in range(args.epochs):
    t = time.time()
    print(f"Epoch {epoch + 1}")
    train(epoch, Network, averaging_steps)
    test()
    total_time = time.time() - t
    print('Total', round(total_time, 2), 's')

Epoch 1

Test set: Average loss: 0.6056, Accuracy: 8234/10000 (82%)

Total 78.91 s
Epoch 2

Test set: Average loss: 0.6962, Accuracy: 8172/10000 (82%)

Total 79.17 s
Epoch 3

Test set: Average loss: 4.0111, Accuracy: 6894/10000 (69%)

Total 79.11 s
Epoch 4

Test set: Average loss: 47.2475, Accuracy: 5340/10000 (53%)

Total 77.84 s
Epoch 5

Test set: Average loss: 969872.6016, Accuracy: 4107/10000 (41%)

Total 78.7 s
Epoch 6

Test set: Average loss: 3675392.0000, Accuracy: 1000/10000 (10%)

Total 79.4 s
Epoch 7

Test set: Average loss: 337267.2000, Accuracy: 1000/10000 (10%)

Total 80.21 s
Epoch 8

Test set: Average loss: 70452.8000, Accuracy: 1000/10000 (10%)

Total 79.4 s
Epoch 9

Test set: Average loss: 67236.0000, Accuracy: 1000/10000 (10%)

Total 80.22 s
Epoch 10

Test set: Average loss: 3.7125, Accuracy: 1000/10000 (10%)

Total 79.04 s


In [15]:
if (args.save_model):
    torch.save(model.state_dict(), "fmnist_cnn.pt")

In [16]:
Network

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [17]:
Network.conv1.weight

Parameter containing:
tensor([[[[ 5.8732e-03, -3.9504e-02, -1.7870e-01, -7.4191e-02, -2.9214e-01],
          [-1.3426e-01, -1.5700e-01, -8.3437e-03, -3.3417e-02, -1.6261e-01],
          [-9.5939e-02, -1.0848e-01, -2.0675e-02, -2.5487e-01, -1.3515e-01],
          [-7.1323e-02, -4.4504e-02, -8.0342e-02, -1.2718e-02, -1.0129e-01],
          [-1.5874e-01, -1.3317e-01, -1.2187e-01, -2.9989e-01, -2.3970e-01]]],


        [[[ 7.6431e-01,  6.7361e-01,  1.7797e-01,  6.1387e-01,  1.7025e+00],
          [ 6.3950e-01,  2.8797e-01, -2.8400e-02,  1.2915e-01,  2.1766e-01],
          [ 2.2663e-01,  2.6438e-01,  2.0695e-01,  6.2982e-01,  5.0779e-01],
          [ 8.4172e-01,  6.6845e-02,  3.8272e-01,  6.9563e-01,  3.8017e-01],
          [ 7.3347e-01,  2.6672e-01,  7.6156e-01,  9.9026e-01,  6.2184e-01]]],


        [[[-8.2168e-01,  5.2884e-02,  4.9734e-01,  9.4060e-01,  4.9069e-01],
          [-7.7437e-01, -6.3548e-02,  2.3213e-01,  7.2448e-01,  6.0773e-01],
          [-8.3059e-01,  3.0314e-01,  2.2814e-

In [18]:
Network.conv1.bias

Parameter containing:
tensor([-2.5173e+01,  4.5732e-01, -2.6506e+00, -2.1839e+00,  4.3497e+00,
        -1.0616e+01,  1.1441e-01, -2.0586e+00, -5.9740e+00, -3.4640e+00,
         8.1539e+00, -1.2343e+00, -8.9784e-01, -3.3684e+00, -7.8274e+00,
        -5.6764e+00, -7.3755e+00,  4.6262e+02, -8.1535e-01,  2.9767e+00],
       device='cuda:0', requires_grad=True)

In [19]:
Network.conv1.bias.grad

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0', grad_fn=<AddBackward0>)